In [ ]:
import cv2
import numpy as np

In [ ]:
def deskew_image_preserve_color(image_path, output_path):
    color_img = cv2.imread(image_path)
    if color_img is None:
        raise ValueError(f"Could not read the image at {image_path}")
    
    gray = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
    
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if len(approx) == 4: 
            corners = approx
            break
    else:
        print("No paper-like structure detected.")
        return
    
    pts = corners.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  
    rect[2] = pts[np.argmax(s)]  

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  
    rect[3] = pts[np.argmax(diff)]  
    

    (tl, tr, br, bl) = rect
    width = max(int(np.linalg.norm(br - bl)), int(np.linalg.norm(tr - tl)))
    height = max(int(np.linalg.norm(tr - br)), int(np.linalg.norm(tl - bl)))

    dst = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype="float32")


    M = cv2.getPerspectiveTransform(rect, dst)
    

    deskewed = cv2.warpPerspective(color_img, M, (width, height))


    cv2.imwrite(output_path, deskewed)

In [ ]:
input_image = 'D:\\Deskew_Images_Code\\Crop_Results\\Image 10.png'
output_image = 'D:\\Deskew_Images_Code\\Results\\Image 10.png'
deskew_image_preserve_color(input_image, output_image)